In [1]:
import os
import gc
import math
import json
from datetime import datetime
import shutil
from pathlib import Path
import optuna
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from tqdm import tqdm
import torch
from sklearn.model_selection import KFold, GroupKFold, StratifiedGroupKFold
from torch.utils.data import DataLoader
from transformers import BatchEncoding, BertTokenizerFast
from typing import Dict, List, Tuple, NamedTuple
import scml
import mylib

In [2]:
class ModelConf(NamedTuple):
    directory: Path
    model_max_length: int
    stride: int
    batch_size: int
    gradient_checkpointing: bool


class Conf(NamedTuple):
    sample_frac: float = 5e-3
    epochs: int = 1
    lr: Tuple[float, float] = (5e-4, 5e-4)
    multi_sample_dropout_size: Tuple[int, int] = (8, 8)
    multi_sample_dropout_increment: Tuple[float, float] = (0, 0)
    swa_start_epoch: Tuple[int, int] = (-1, -1)
    swa_anneal_epochs: Tuple[int, int] = (3, 3)
    pretrained_dir: Path = Path("pretrained")
    hidden_size: int = 256
    model_max_length: int = 32
    stride: int = 0
    batch_size: int = 4
    model_name: str = "roberta_to_roberta"
    gradient_checkpointing: bool = False
    gpus: List[int] = [0]
    patience: int = 0
    n_trials: int = 1
    n_folds: int = 3
    seed: int = 31
    
        
conf = Conf()
print(conf)

Conf(sample_frac=0.005, epochs=1, lr=(0.0005, 0.0005), multi_sample_dropout_size=(8, 8), multi_sample_dropout_increment=(0, 0), swa_start_epoch=(-1, -1), swa_anneal_epochs=(3, 3), pretrained_dir=WindowsPath('pretrained'), hidden_size=256, model_max_length=32, stride=0, batch_size=4, model_name='roberta_to_roberta', gradient_checkpointing=False, gpus=[0], patience=0, n_trials=1, n_folds=3, seed=31)


In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(conf.seed)
pl.seed_everything(conf.seed)

Global seed set to 31


31

In [4]:
device: torch.device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce GTX 1060 6GB
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [5]:
with open("input/vocab2.json") as f:
    id2label = json.load(f)
print(f"len(id2label)={len(id2label):,}\nid2label[:10]={id2label[:10]}")

len(id2label)=3,748,278
id2label[:10]=[['1460571_click', 121287], ['108125_click', 114456], ['29735_click', 101148], ['485256_click', 97154], ['1733943_click', 91395], ['184976_click', 85122], ['832192_click', 81127], ['1502122_click', 73805], ['554660_click', 72161], ['1603001_click', 68519]]


In [6]:
df = pd.read_parquet("input/pairs.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14461968 entries, 0 to 14461967
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   center   int32
 1   outside  int32
dtypes: int32(2)
memory usage: 110.3 MB


In [7]:
ds = mylib.SkipGramDataset(
    center_words=df["center"].tolist(), 
    outside_words=df["outside"].tolist(),
    #session_ids=sids,
)
print(f"len(ds)={len(ds):,}\n{ds[30]}\n{ds[0]}")

len(ds)=14,461,968
{'center_words': tensor(154), 'outside_words': tensor(101)}
{'center_words': tensor(3), 'outside_words': tensor(130483)}


In [8]:
del df
gc.collect()
job_ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = Path("models") / conf.model_name / job_ts
job_dir.mkdir(parents=True, exist_ok=True)
print(f"job_dir={job_dir}")

job_dir=models\roberta_to_roberta\20230116_084458


# Train final model on best Hps

In [9]:
%%time
#splitter = GroupKFold(n_splits=50)
splitter = KFold(n_splits=50)
dummy = np.zeros(len(ds))
#for ti, vi in splitter.split(dummy, y=ds.stratification(), groups=ds.groups()):
for ti, vi in splitter.split(dummy):
    tra_ds = torch.utils.data.Subset(ds, ti)
    val_ds = torch.utils.data.Subset(ds, vi)
    break
print(f"len(tra_ds)={len(tra_ds)}, len(val_ds)={len(val_ds)}")

len(tra_ds)=14172728, len(val_ds)=289240
Wall time: 56.9 ms


In [10]:
#best = df.iloc[0].to_dict()
best = {
    "lr": conf.lr[0],
}
print(f"best={best}")

best={'lr': 0.0005}


In [11]:
model = mylib.Word2VecLightningModel(
    lr=best["lr"],
    vocab_size=len(id2label),
    embedding_size=128,
    negative_samples=10,
)
print(model.model)

SkipGramWord2Vec(
  (embeddings): Embedding(3748278, 128)
)


In [12]:
gc.collect()
torch.cuda.empty_cache()
trainer = mylib.Trainer(
    default_root_dir=job_dir,
    gpus=conf.gpus,
    max_epochs=conf.epochs,  
    callbacks=mylib.training_callbacks(patience=conf.patience),
    deterministic=False,
)
trainer.fit(
    model,
    train_dataloaders=DataLoader(
        tra_ds,
        batch_size=conf.batch_size,
        shuffle=True,
        num_workers=0,
    ),
    val_dataloaders=DataLoader(
        val_ds,
        batch_size=conf.batch_size,
        shuffle=False,
        num_workers=0,
    ),
)

s:\dev\seahrh\kaggle-otto-recommender-system-2022\env\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: models\roberta_to_roberta\20230116_084458\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | SkipGramWord2Vec | 479 M 
-------------------------------------------
479 M     Trainable params
0         Non-trainable params
479 M     Total params
1,919.118 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[INFO|word2vec.py:59] 2023-01-16 08:45:06,251 >> center_word.size=torch.Size([4]), outside_word.size=torch.Size([4])
[INFO|word2vec.py:59] 2023-01-16 08:45:06,251 >> center_word.size=torch.Size([4]), outside_word.size=torch.Size([4])


s:\dev\seahrh\kaggle-otto-recommender-system-2022\env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")
print(f"Saved {str(job_dir)}")